In [1]:
import pandas as pd
import numpy as np
from fastai.vision import *
from fastai.core import Category
from pathlib import Path
from torch.nn import DataParallel

from sklearn.model_selection import train_test_split
from skimage.transform import resize

In [2]:
# 测试集没有旋转，所以可以直接删掉，而且旋转的npy的长宽都有问题
train2 = pd.read_csv('train2.csv')[lambda x: ((x.ymax - x.ymin) <= (x.xmax - x.xmin)) & (x.xmin != 200)]
train2['filename'] = '/mnt/bak/ga/train2/' + train2.index.astype(str) + '_2.npy'
train2 = train2.reset_index(drop=True)

BS_SIZE = 32
NUM_WORKERS = 8

In [4]:
train_idx, val_idx = train_test_split(range(len(train2)), test_size=0.2, random_state=666, stratify=train2.ret)

In [5]:
def open_npy(self, fname):
    x = np.load(fname).astype(np.float32) / 255
    x = torch.from_numpy(x)
    return Image(x)
ImageList.open=open_npy
tfms = get_transforms(do_flip=False)
data = (ImageList.from_df(path='/', df=train2, cols='filename')
    .split_by_idxs(train_idx, val_idx)
    .label_from_df('ret')
    .add_test(ImageList.from_df(path='/', df=train2, cols='filename'))
    .transform(tfms, size=(21, 320, 400), padding_mode='zeros')
    .databunch(bs=BS_SIZE, num_workers=NUM_WORKERS)
    .normalize())
data

ImageDataBunch;

Train: LabelList (3120 items)
x: ImageList
Image (21, 320, 400),Image (21, 320, 400),Image (21, 320, 400),Image (21, 320, 400),Image (21, 320, 400)
y: CategoryList
0,0,0,1,0
Path: /;

Valid: LabelList (781 items)
x: ImageList
Image (21, 320, 400),Image (21, 320, 400),Image (21, 320, 400),Image (21, 320, 400),Image (21, 320, 400)
y: CategoryList
0,1,0,1,0
Path: /;

Test: LabelList (3901 items)
x: ImageList
Image (21, 320, 400),Image (21, 320, 400),Image (21, 320, 400),Image (21, 320, 400),Image (21, 320, 400)
y: EmptyLabelList
,,,,
Path: /

In [6]:
class Dummy(nn.Module):
    "Just returns the mean of the `output`."
    def forward(self, output, *args): return output

# resnet34
class CombineAllChannelsNet(nn.Module):
    def __init__(self):
        super(CombineAllChannelsNet, self).__init__()
        body = create_body(models.resnet34)
        body_out_features = 512
        body[0].weight.data = body[0].weight.data[:, 0].unsqueeze(1)
        body[0].in_channels = 1
        self.body = body
        self.neck = Dummy()
        self.head = create_head(21*body_out_features*2, 2, [1024])  # 之前的输出是512，21个channels，每个有两个输出，所以是21*1024

    def forward(self, inp):
        lst_oup = []
        for i in range(21):
            lst_oup.append(self.body(inp[:, i].unsqueeze(1)))
        combined = torch.cat(lst_oup, dim=1)
        combined = self.neck(combined)
        out = self.head(combined)
        return out
    
model = CombineAllChannelsNet()
apply_init(model.head, nn.init.kaiming_normal_)
learn = Learner(data, model, metrics=[accuracy, Precision(), Recall()], path='/home/ubuntu/ga/model/')
learn.model

CombineAllChannelsNet(
  (body): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [7]:
m = learn.model.eval();